In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import warnings
warnings.filterwarnings("ignore")
from spacy.tokens import DocBin
import spacy
from tqdm import tqdm

In [5]:
df_train=pd.read_csv('final_train.csv', encoding='utf-8', encoding_errors='ignore')
df_train.columns=['sentence','tags']

df_test=pd.read_csv('final_test.csv', encoding='utf-8', encoding_errors='ignore')
df_test.columns=['sentence','tags']

In [7]:
# function to obtain entity format according to Spacy
def get_entity_tuple(df):
  list_of_entities = []
  for x in range(len(df)):
    d = {"entities": []}
    i, j = 0, 0
    idx = -1
    
    tokens = df["tags"][x].split()
    words = df['sentence'][x].split()
    if len(words) != len(tokens):
      continue

    temp = df["sentence"][x] + " "
    for char in temp:
      if char == " ":
        d["entities"].append((i, j, tokens[idx]))
        j += 1
        i = j
        idx += 1
      else:
        j += 1
    list_of_entities.append((df["sentence"][x], d))
  
  return list_of_entities

In [8]:
# apply on train and test set
train_entities = get_entity_tuple(df_train)
test_entities = get_entity_tuple(df_test)

6611
[('2 tablespoons vegetable oil , divided', {'entities': [(0, 1, 'STATE'), (2, 13, 'QUANTITY'), (14, 23, 'UNIT'), (24, 27, 'NAME'), (28, 29, 'NAME'), (30, 37, 'O')]}), ('2 tablespoons dried marjoram', {'entities': [(0, 1, 'NAME'), (2, 13, 'QUANTITY'), (14, 19, 'UNIT'), (20, 28, 'DF')]})]
2187
[('6 ounces fresh mushrooms , sliced', {'entities': [(0, 1, 'STATE'), (2, 8, 'QUANTITY'), (9, 14, 'UNIT'), (15, 24, 'DF'), (25, 26, 'NAME'), (27, 33, 'O')]}), ('4 corn tortillas', {'entities': [(0, 1, 'NAME'), (2, 6, 'QUANTITY'), (7, 16, 'NAME')]})]


In [14]:
db = DocBin()
nlp = spacy.load("en_core_web_sm")

for text, annot in tqdm(train_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./train.spacy") # save the docbin object


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6611/6611 [00:12<00:00, 522.36it/s]


Skipping entity


In [15]:
db = DocBin()
nlp = spacy.load("en_core_web_sm")

for text, annot in tqdm(test_entities): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)
    
db.to_disk("./test.spacy") # save the docbin object

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2187/2187 [00:00<00:00, 6043.65it/s]


Skipping entity


In [11]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2023-02-14 15:04:04.618271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-02-14 15:04:04.618606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 15:04:26.471750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-02-14 15:04:26.473079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2023-02-14 15:04:26.476216: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2023-02-14 15:04:26.478751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy